In [1]:
import wandb
from collections import defaultdict
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from IPython import embed

In [2]:
api = wandb.Api()

In [3]:
# List of tags to identify the set of sweeps in this analysis

TAGS = ["feats8"]

In [25]:
runs = api.runs("dhdhagar/prob-ent-resolution",
                filters={"tags": {"$in": TAGS}, 
                         "state": {"$eq": "finished"},
                         "config.pairwise_mode": {"$eq": False}
                        },
                order="-summary_metrics.best_dev_b3_f1"
               )
runs_mlp = api.runs("dhdhagar/prob-ent-resolution", 
                filters={"tags": {"$in": TAGS}, 
                         "state": {"$eq": "finished"},
                         "config.pairwise_mode": {"$eq": True}
                        },
                order="-summary_metrics.best_dev_auroc"
               )

In [26]:
len(runs)

5105

In [27]:
len(runs_mlp)

2162

In [7]:
methods = {'e2e', 'e2e-nosdp', 'frac', 'frac-nosdp', 'mlp'}
def run_key(run):
    dataset = run._attrs['config']['dataset']
    dataset_seed = run._attrs['config']['dataset_random_seed']
    method = set(run.tags).intersection(methods).pop()
    key = f"{method}_{dataset}_{dataset_seed}"
    return key

In [8]:
best_runs = {}
finished_runs = defaultdict(int)
all_sweeps = {}
for _runs in [runs, runs_mlp]:
    for run in tqdm(_runs):
        if len(dict(run.summary)) < 10:
            continue
        key = run_key(run)
        finished_runs[key] += 1
        if key in best_runs:
            continue
        best_runs[key] = run
        all_sweeps[key] = run.sweepName

  0%|          | 0/5105 [00:00<?, ?it/s]

  0%|          | 0/2162 [00:00<?, ?it/s]

In [9]:
print(f"Best runs found for {len(best_runs)} sweeps")
print(f"Total finished runs across sweeps = {sum(finished_runs.values())}")

Best runs found for 75 sweeps
Total finished runs across sweeps = 7264


# Analyze results

In [28]:
res_map = {
    'train_time': 'z_run_time',
    'inf_time_hac': 'z_inf_time_hac',
    'inf_time_cc': 'z_inf_time_cc',
    'inf_time_cc-nosdp': 'z_inf_time_cc-nosdp',
    'b3_f1_hac': 'best_test_b3_f1_hac',
    'b3_f1_cc': 'best_test_b3_f1_cc',
    'b3_f1_cc-fixed': 'best_test_b3_f1_cc-fixed',
    'b3_f1_cc-nosdp': 'best_test_b3_f1_cc-nosdp',
    'b3_f1_cc-nosdp-fixed': 'best_test_b3_f1_cc-nosdp-fixed',
    'vmeasure_hac': 'best_test_vmeasure_hac',
    'vmeasure_cc': 'best_test_vmeasure_cc',
    'vmeasure_cc-fixed': 'best_test_vmeasure_cc-fixed',
    'vmeasure_cc-nosdp': 'best_test_vmeasure_cc-nosdp',
    'vmeasure_cc-nosdp-fixed': 'best_test_vmeasure_cc-nosdp-fixed'
}

In [29]:
final = {}
for run_id, run in best_runs.items():
    _key = run_id[:-2]  # Remove the seed
    if _key not in final:
        final[_key] = defaultdict(list)
    res = dict(run.summary)
    for out_key, in_key in res_map.items():
        final[_key][out_key].append(float(res[in_key]))
means, stds, comb = {}, {}, {}
for k in final:
    if k is not means:
        means[k] = {}
        stds[k] = {}
        comb[k] = {}
    for _k in final[k]:
        means[k][_k] = round(np.mean(final[k][_k])*(1 if 'time' in _k else 100), 2)
        stds[k][_k] = round(np.std(final[k][_k])*(1 if 'time' in _k else 100), 2)
        comb[k][_k] = f"{means[k][_k]}±{stds[k][_k]}"

In [30]:
def get_df_by_dataset(res, dataset, to_latex=False):
    new_res = {}
    for _r in res:
        if dataset in _r:
            new_res[_r.replace(f"{dataset}_", '').replace(f"_{dataset}", '')] = res[_r]
    if to_latex:
        print(pd.DataFrame(new_res).T.style.to_latex())
    outdf = pd.DataFrame(new_res).T.sort_index()
    
    def highlight_max(s):
        if s.dtype == object:
            is_max = [False for _ in range(s.shape[0])]
            if '±' in s[0]:
                nums = np.array(list(map(lambda x: float(x.split('±')[0]), s)))
                is_max = nums == nums.max()
        else:
            is_max = s == s.max()
        return ['color: green' if cell else '' for cell in is_max]
    
    if outdf[outdf.keys()[0]].dtype == object:
        return outdf.style.apply(highlight_max), outdf
    return outdf.style.format('{:.2f}').apply(highlight_max), outdf

In [31]:
outdf, _outdf = get_df_by_dataset(comb, 'pubmed')
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,1259.0±428.11,39.2±17.06,33.8±8.3,13.6±1.62,77.94±3.67,82.82±2.36,69.52±23.27,82.3±1.71,68.96±23.0,88.57±2.64,91.3±1.83,83.42±14.35,91.18±1.97,83.35±14.35
e2e-nosdp,234.0±21.57,31.6±13.48,21.4±10.59,11.4±2.06,85.92±3.6,82.7±3.92,81.54±3.99,85.3±3.01,84.06±4.05,93.14±1.91,90.9±2.86,90.1±3.13,92.84±1.72,91.88±2.71
frac,814.0±138.06,31.2±14.96,23.2±11.69,19.8±20.13,79.86±3.68,83.98±1.43,81.14±1.67,82.56±3.24,81.08±1.85,89.67±0.91,91.83±0.95,90.12±2.46,91.24±2.48,90.5±1.78
frac-nosdp,317.6±108.19,62.4±44.7,29.0±7.92,19.0±6.69,62.78±9.03,77.82±2.81,60.82±20.39,79.18±3.04,60.54±20.27,80.16±5.63,88.21±1.85,75.97±16.18,89.41±2.02,75.72±16.02
mlp,279.8±78.41,32.4±15.16,19.0±8.72,10.6±3.38,83.32±4.18,80.58±2.62,78.28±5.23,82.06±2.46,77.64±5.67,91.24±2.11,89.67±1.37,87.91±3.64,90.94±1.86,88.06±3.64


In [32]:
outdf, _outdf = get_df_by_dataset(comb, 'qian')
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,1936.8±1327.21,146.4±157.89,32.2±23.46,24.0±16.41,69.58±6.43,69.2±6.61,66.84±7.47,69.76±6.96,66.98±7.56,84.32±4.32,84.2±4.39,82.8±5.17,84.55±4.53,82.88±5.2
e2e-nosdp,256.4±28.32,57.0±22.42,24.2±13.99,11.2±1.47,72.88±5.15,73.08±3.9,68.24±8.07,73.62±5.58,68.94±8.0,85.69±4.17,86.02±3.35,84.7±4.13,86.11±4.39,85.17±3.95
frac,1600.0±671.38,123.2±143.4,21.0±6.07,11.6±3.93,70.14±6.22,70.3±6.58,62.96±8.45,70.92±7.05,62.8±9.59,84.31±3.99,84.32±4.05,81.71±4.74,84.78±4.34,81.97±5.08
frac-nosdp,290.8±110.22,79.2±18.86,26.0±13.61,13.8±7.6,69.28±6.02,71.82±5.35,57.24±7.07,71.94±5.47,57.02±7.83,83.86±4.1,85.12±4.23,80.56±4.21,85.37±4.05,81.18±3.74
mlp,275.4±97.82,53.8±26.48,20.8±11.5,12.8±7.68,73.28±5.4,52.42±13.75,45.66±17.08,52.46±13.98,45.84±17.16,86.03±3.87,80.84±4.66,79.16±5.29,81.19±4.48,79.34±5.26


In [33]:
outdf, _outdf = get_df_by_dataset(comb, 'arnetminer')
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,1738.4±627.6,129.4±128.63,76.2±71.62,12.0±2.19,66.5±10.78,59.76±10.06,55.34±9.67,60.26±10.61,54.66±10.12,78.33±7.29,77.03±7.12,75.35±7.34,77.3±7.17,75.37±7.23
e2e-nosdp,143.2±13.41,73.4±23.47,69.8±67.15,13.6±2.15,67.34±11.12,65.36±9.74,63.74±8.72,66.64±10.6,63.36±8.6,78.91±7.6,78.09±7.11,77.64±6.9,78.76±7.36,77.83±7.16
frac,1208.0±892.75,123.8±54.03,35.8±23.79,11.2±2.56,58.82±13.93,57.88±12.69,58.36±11.54,58.02±12.91,57.78±11.75,76.36±5.15,76.02±4.7,73.47±6.24,76.02±4.76,73.49±6.27
frac-nosdp,138.4±17.32,201.2±277.46,67.4±75.15,16.8±3.54,64.68±11.87,59.86±11.57,51.68±6.78,60.5±12.08,50.74±6.83,77.77±6.91,77.85±5.42,75.19±4.62,78.25±5.64,74.86±4.81
mlp,310.2±18.89,67.0±23.35,53.4±44.56,11.4±1.02,67.84±9.65,59.54±8.29,58.4±9.72,60.04±8.14,57.88±9.5,80.52±5.95,77.52±5.79,76.06±4.76,78.07±5.71,76.01±4.9


# Manage sweeps

In [10]:
SWEEP_PREFIX="feats8"
SWEEP_FLAGS = "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2 \
    --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5 \
    --keep_feat_idxs=14 --keep_feat_idxs=15"

In [11]:
# Add agents to sweeps with fewer completed runs

def add_agents(max_run_count_filter, already_added, sweep_finished_count, sweep_ids, n_agents=2, gpu="gypsum-1080ti"):
    for k, v in sweep_finished_count.items():
        if k in already_added:
            continue
        if v < max_run_count_filter:
            model, dataset, seed = k.split('_')
            sweep_id = sweep_ids[k]
            n_agents = n_agents
            gpu_name = gpu
            print(f'./add_agent.sh {dataset} {seed} {model} dhdhagar/prob-ent-resolution/{sweep_id} {n_agents} {gpu_name} "{SWEEP_FLAGS}" {SWEEP_PREFIX}')
            !./add_agent.sh $dataset $seed $model dhdhagar/prob-ent-resolution/$sweep_id $n_agents $gpu_name "$SWEEP_FLAGS" $SWEEP_PREFIX
            already_added.add(k)

In [12]:
def add_agents_by_list(keys_to_add, already_added, sweep_ids, n_agents=2, gpu="gypsum-1080ti"):
    for k in keys_to_add:
        model, dataset, seed = k.split('_')
        sweep_id = sweep_ids[k]
        n_agents = n_agents
        gpu_name = gpu
        print(f'./add_agent.sh {dataset} {seed} {model} dhdhagar/prob-ent-resolution/{sweep_id} {n_agents} {gpu_name} "{SWEEP_FLAGS}" {SWEEP_PREFIX}')
        !./add_agent.sh $dataset $seed $model dhdhagar/prob-ent-resolution/$sweep_id $n_agents $gpu_name "$SWEEP_FLAGS" $SWEEP_PREFIX
        already_added.add(k)

In [13]:
def add_sweeps_by_list(keys_to_add, already_added, gpu="gypsum-1080ti"):
    for k in keys_to_add:
        model, dataset, seed = k.split('_')
        gpu_name = gpu
        print(f'./add_agent.sh {dataset} {seed} {seed} {model} {gpu_name} "{SWEEP_FLAGS}" {SWEEP_PREFIX}')
        !./run_sweep.sh $dataset $seed $seed $model $gpu_name "$SWEEP_FLAGS" $SWEEP_PREFIX
        already_added.add(k)

In [14]:
def resubmit_pending_agents(already_added, sweep_ids, n_agents=2, gpu="gypsum-1080ti"):
    pending = !sacct --format="JobID,JobName%50,Partition,State" | grep PENDING
    print('\n'.join(pending))
    _pending_agents = set()
    _pending_agent_ids = []
    for pen in pending:
        pen_split = pen.split()
        model, dataset, seed, _ = pen_split[1].split('_')
        seed_split = seed.split('-')
        seed = seed_split[0][-1]
        if len(seed_split) == 1:
            # Sweep init job is pending
            continue
        else:
            # Sweep agent job is pending
            _pending_agents.add(f"{model}_{dataset}_{seed}")
            _pending_agent_ids.append(pen_split[0])
    print(f"PENDING agents: {_pending_agents}\n")
    if len(_pending_agents) > 0:
        _pending_agent_ids = ' '.join(_pending_agent_ids)
        print(f"!scancel {_pending_agent_ids}")
        !scancel $_pending_agent_ids
        add_agents_by_list(keys_to_add=_pending_agents, already_added=already_added,
                           sweep_ids=sweep_ids, n_agents=n_agents, gpu=gpu)

In [15]:
def resubmit_pending_sweeps(already_added, gpu="gypsum-1080ti"):
    pending = !sacct --format="JobID,JobName%50,Partition,State" | grep PENDING
    print('\n'.join(pending))
    _pending_sweeps = set()
    _pending_sweep_ids = []
    for pen in pending:
        pen_split = pen.split()
        model, dataset, seed, _ = pen_split[1].split('_')
        seed_split = seed.split('-')
        seed = seed_split[0][-1]
        if len(seed_split) == 1:
            # Sweep init job is pending
            _pending_sweeps.add(f"{model}_{dataset}_{seed}")
            _pending_sweep_ids.append(pen_split[0])
        else:
            # Sweep agent job is pending
            continue
    print(f"PENDING sweeps: {_pending_sweeps}\n")
    if len(_pending_sweeps) > 0:
        _pending_sweep_ids = ' '.join(_pending_sweep_ids)
        print(f"!scancel {_pending_sweep_ids}")
        !scancel $_pending_sweep_ids
        add_sweeps_by_list(keys_to_add=_pending_sweeps, already_added=already_added, gpu=gpu)

In [36]:
agents_added = set()

In [37]:
add_agents(max_run_count_filter=60, already_added=agents_added, 
           sweep_finished_count=finished_runs, sweep_ids=all_sweeps, 
           n_agents=2, gpu="cpu")

./add_agent.sh pubmed 3 e2e dhdhagar/prob-ent-resolution/nr853f9c 2 cpu "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5     --keep_feat_idxs=14 --keep_feat_idxs=15" feats8
Submitted batch job 6776831
    Logs: jobs/e2e_pubmed_sweep3-agent1_1680985411.err
Submitted batch job 6776832
    Logs: jobs/e2e_pubmed_sweep3-agent2_1680985411.err
./add_agent.sh pubmed 2 e2e dhdhagar/prob-ent-resolution/w46o4bx8 2 cpu "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5     --keep_feat_idxs=14 --keep_feat_idxs=15" feats8
Submitted batch job 6776833
    Logs: jobs/e2e_pubmed_sweep2-agent1_1680985411.err
Submitted batch job 6776834
    Logs: jobs/e2e_pubmed_sweep2-agent2_1680985411.err
./add_agent.sh pubmed 1 frac-nosdp dhdhagar/prob-ent-resolution/mwzvxsd0 2 cpu "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --k

Submitted batch job 6776873
    Logs: jobs/mlp_arnetminer_sweep3-agent2_1680985417.err


In [ ]:
add_agents(max_run_count_filter=30, already_added=agents_added, 
           sweep_finished_count=finished_runs, sweep_ids=all_sweeps, 
           n_agents=2, gpu="gypsum-1080ti")

In [29]:
add_agents(max_run_count_filter=30, already_added=agents_added, 
           sweep_finished_count=finished_runs, sweep_ids=all_sweeps, 
           n_agents=2, gpu="gypsum-2080ti")

./add_agent.sh pubmed 3 e2e dhdhagar/prob-ent-resolution/nr853f9c 2 gypsum-2080ti "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5     --keep_feat_idxs=14 --keep_feat_idxs=15" feats8
Submitted batch job 6748713
    Logs: jobs/e2e_pubmed_sweep3-agent1_1680933250.err
Submitted batch job 6748714
    Logs: jobs/e2e_pubmed_sweep3-agent2_1680933250.err
./add_agent.sh pubmed 1 e2e dhdhagar/prob-ent-resolution/bg6l5u1j 2 gypsum-2080ti "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5     --keep_feat_idxs=14 --keep_feat_idxs=15" feats8
Submitted batch job 6748715
    Logs: jobs/e2e_pubmed_sweep1-agent1_1680933250.err
Submitted batch job 6748716
    Logs: jobs/e2e_pubmed_sweep1-agent2_1680933250.err
./add_agent.sh pubmed 2 e2e dhdhagar/prob-ent-resolution/w46o4bx8 2 gypsum-2080ti "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3

In [30]:
add_agents(max_run_count_filter=40, already_added=agents_added, 
           sweep_finished_count=finished_runs, sweep_ids=all_sweeps, 
           n_agents=2, gpu="gypsum-titanx")

./add_agent.sh pubmed 3 frac dhdhagar/prob-ent-resolution/3j18jnrp 2 gypsum-titanx "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5     --keep_feat_idxs=14 --keep_feat_idxs=15" feats8
Submitted batch job 6748729
    Logs: jobs/frac_pubmed_sweep3-agent1_1680933253.err
Submitted batch job 6748730
    Logs: jobs/frac_pubmed_sweep3-agent2_1680933253.err
./add_agent.sh pubmed 2 frac dhdhagar/prob-ent-resolution/1py4a59t 2 gypsum-titanx "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5     --keep_feat_idxs=14 --keep_feat_idxs=15" feats8
Submitted batch job 6748731
    Logs: jobs/frac_pubmed_sweep2-agent1_1680933253.err
Submitted batch job 6748732
    Logs: jobs/frac_pubmed_sweep2-agent2_1680933253.err
./add_agent.sh pubmed 4 frac dhdhagar/prob-ent-resolution/uuruuipk 2 gypsum-titanx "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat

In [31]:
add_agents(max_run_count_filter=50, already_added=agents_added, 
           sweep_finished_count=finished_runs, sweep_ids=all_sweeps, 
           n_agents=2, gpu="gpu")

./add_agent.sh pubmed 2 frac-nosdp dhdhagar/prob-ent-resolution/uvjk8ltp 2 gpu "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5     --keep_feat_idxs=14 --keep_feat_idxs=15" feats8
Submitted batch job 6748752
    Logs: jobs/frac-nosdp_pubmed_sweep2-agent1_1680933256.err
Submitted batch job 6748753
    Logs: jobs/frac-nosdp_pubmed_sweep2-agent2_1680933256.err
./add_agent.sh pubmed 1 frac dhdhagar/prob-ent-resolution/79z8xtk8 2 gpu "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5     --keep_feat_idxs=14 --keep_feat_idxs=15" feats8
Submitted batch job 6748754
    Logs: jobs/frac_pubmed_sweep1-agent1_1680933257.err
Submitted batch job 6748755
    Logs: jobs/frac_pubmed_sweep1-agent2_1680933257.err
./add_agent.sh arnetminer 2 frac dhdhagar/prob-ent-resolution/cl1bjd96 2 gpu "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 

In [32]:
resubmit_pending_agents(already_added=agents_added, sweep_ids=all_sweeps,
                 n_agents=2, gpu="gypsum-titanx")

6748771                       mlp_qian_sweep3-agent1_1680933258        gpu    PENDING 
6748772                       mlp_qian_sweep3-agent2_1680933258        gpu    PENDING 
6748773                       mlp_qian_sweep2-agent1_1680933258        gpu    PENDING 
6748774                       mlp_qian_sweep2-agent2_1680933258        gpu    PENDING 
6748775                       mlp_qian_sweep1-agent1_1680933259        gpu    PENDING 
6748776                       mlp_qian_sweep1-agent2_1680933259        gpu    PENDING 
PENDING agents: {'mlp_qian_2', 'mlp_qian_1', 'mlp_qian_3'}
!scancel 6748771 6748772 6748773 6748774 6748775 6748776
./add_agent.sh qian 2 mlp dhdhagar/prob-ent-resolution/gcteakms 2 gypsum-titanx "--keep_feat_idxs=0 --keep_feat_idxs=1 --keep_feat_idxs=2     --keep_feat_idxs=3 --keep_feat_idxs=4 --keep_feat_idxs=5     --keep_feat_idxs=14 --keep_feat_idxs=15" feats8
Submitted batch job 6748851
    Logs: jobs/mlp_qian_sweep2-agent1_1680933409.err
Submitted batch job 6748852
  

In [33]:
resubmit_pending_agents(already_added=agents_added, sweep_ids=all_sweeps,
                 n_agents=2, gpu="gypsum-titanx")


PENDING agents: set()
